In [47]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, IPython
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [4]:
"""
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
from sagemaker.predictor import csv_serializer
if int(sagemaker.__version__.split('.')[0]) == 2:
    print("Installing previous SageMaker Version and restarting the kernel")
    !{sys.executable} -m pip install sagemaker==1.72.0
    IPython.Application.instance().kernel.do_shutdown(True)

else:
    print("Version is good")
    
role = get_execution_role()
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')
# create a s3 bucket to store model related stuff
bucket_name = 'airplane-ticket-model'
s3 = boto3.resource('s3')
try:
    if  region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)
"""

'\nfrom sagemaker import get_execution_role\nfrom sagemaker.analytics import ExperimentAnalytics\nfrom smexperiments.experiment import Experiment\nfrom smexperiments.trial import Trial\nfrom smexperiments.trial_component import TrialComponent\nfrom smexperiments.tracker import Tracker\nfrom sagemaker.predictor import csv_serializer\nif int(sagemaker.__version__.split(\'.\')[0]) == 2:\n    print("Installing previous SageMaker Version and restarting the kernel")\n    !{sys.executable} -m pip install sagemaker==1.72.0\n    IPython.Application.instance().kernel.do_shutdown(True)\n\nelse:\n    print("Version is good")\n    \nrole = get_execution_role()\nsess = sagemaker.Session()\nregion = boto3.session.Session().region_name\nprint("Region = {}".format(region))\nsm = boto3.Session().client(\'sagemaker\')\n# create a s3 bucket to store model related stuff\nbucket_name = \'airplane-ticket-model\'\ns3 = boto3.resource(\'s3\')\ntry:\n    if  region == \'us-east-1\':\n        s3.create_bucket(Bu

In [15]:
# import dataset from s3 bucket
bucket='planetickets'
data_key = 'Cleaned_2018_Flights.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)


In [5]:
data = pd.read_csv("../data/Cleaned_2018_Flights.csv")
data = data.dropna()
data = data.drop(columns = ['Unnamed: 0', 'ItinID', 'MktID', 'MktCoupons', 'OriginWac', 'DestWac', 'ContiguousUSA', "Miles"])

# Y: PricePerTicket; X: all other variables
X = data.drop(columns = ['PricePerTicket'])
Y = data[['PricePerTicket']]

# Split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [6]:
numerical_cols = ["NumTicketsOrdered"]
categorical_cols = ['Quarter', "Origin", "Dest", "AirlineCompany"] 

In [9]:
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
            ('cat', categorical_transformer, categorical_cols)], remainder='passthrough')

In [10]:
X_train_transformed = preprocessor.fit_transform(X_train).toarray()

MemoryError: Unable to allocate array with shape (6388059, 533) and data type float64

In [55]:
X_train_transformed[0].sum()

4.0

In [11]:
X_train

,Quarter,Origin,Dest,NumTicketsOrdered,AirlineCompany
4689011,3,DFW,LAS,1.0,AA
4139132,2,SAV,JFK,1.0,B6
9502133,4,LAS,RNO,1.0,WN
4391641,2,AUS,DEN,1.0,UA
4202516,2,ATL,MCO,6.0,DL
...,...,...,...,...,...
9524682,4,LAS,LBB,1.0,WN
2234489,2,GEG,DFW,2.0,AA
4304572,2,SDF,LGA,1.0,DL
6550634,3,ATL,EVV,1.0,DL


In [12]:
data

,Quarter,Origin,Dest,NumTicketsOrdered,AirlineCompany,PricePerTicket
0,1,PHL,LAX,1.0,AA,672.87
1,1,PHL,LAX,1.0,AA,367.68
2,1,PHL,LAX,1.0,AA,417.94
3,1,PHL,LAX,1.0,AA,247.10
4,1,PHL,LAX,1.0,AA,276.35
...,...,...,...,...,...,...
9534412,4,STL,AUS,1.0,WN,310.10
9534413,4,BNA,MCI,1.0,WN,106.17
9534414,4,BOS,LIT,1.0,WN,161.19
9534415,4,BOS,LIT,1.0,WN,170.87
